In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])   
    fig.show()

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_10','F_SMA_20'
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [6]:
Test_Candle = [5397, 5994, 2662, 5437, 6228, 9242, 5740,  706, 3041, 2241, 8006,
               2944, 9326, 7754, 6234,  502, 3246, 3377, 3790, 5753, 2820, 8363,
               5353, 6291, 8374, 3748, 1679, 3447, 2476, 8618, 6665, 6051, 5561,
               9204, 8956, 5777, 2013, 5422, 9474, 8412, 8040, 2496, 6678, 5959,
               3939, 6690, 8488, 4112, 3856, 4580, 6107,  231, 8828, 8438, 3905,
               4586, 8565, 7728, 3821, 3890, 4690, 6765,  804, 7052, 3505, 8203,
               8403, 8476, 2871, 6077, 2908, 9574, 1276,  796, 5751, 7664, 9115,
               2700, 1187, 5162, 5346, 4117, 2183, 8687, 7338, 9482, 1838, 7831,
               3647, 3244,  679, 2726, 4744, 4836, 4629, 3289, 5621, 1123, 1790,
               3287, 3743, 4340,  359, 1790, 3916, 7811, 7029, 9513, 4251, 1193,
               1426, 2675, 1631, 6272, 7134, 1891, 7640, 5836, 1783, 8741, 5635,
               1780, 5715,  743, 9600, 8187, 8825, 1957, 7363, 8282, 3559, 9204,
               1793, 2130, 9160, 1558, 6858, 2090, 8998, 2969, 8812, 1733, 8408,
               6034, 8321, 6986, 5095, 7472,  449, 1650, 3861, 9369, 2741, 4883,
               7806, 3305, 8378, 5242,   52, 3651, 3434, 2585, 8919, 5754, 3691,
               3033, 2438, 1401, 5579, 7130, 5590,  181, 8436, 4850, 4119, 3907,
               7718, 4616, 8113, 3971, 8202, 4699, 3645, 3495, 4655, 8491, 5584,
               2215, 1775,  769,  327, 2163, 5655,  891, 4028,  964, 2739, 9212,
               6511, 7263, 3178, 2180, 5862, 5748, 4956, 2473, 3868, 5485, 3859,
               8649, 3111, 1176, 3277, 3466, 3150, 8303,  317, 1109, 4040, 6397,
               8621, 8397, 7702, 8723, 6734, 5315, 8332, 4329, 4524, 6900,  588,
               3448, 3758, 8594, 9453, 9122, 6297, 1350, 6435, 2546, 7001,  551,
               6548, 2862, 6343, 6211, 1276, 6085, 4956, 1778, 5463, 2950, 3753,
               4012, 5925, 4660, 6116, 2530, 6046, 5474, 2114, 2455, 1813, 3588,
               3843, 5926, 3806, 2665, 4312, 8251, 1011, 2335, 8657, 3105, 1095,
                756, 3605, 5425, 5877, 4805, 8413, 2934,  112, 8310, 1194, 5448,
               8912, 5037, 6763, 5598, 5791, 7206, 8220, 3701, 1932, 7650, 6958,
               8546, 3542, 4000, 2950, 8896, 5334, 2757, 4418, 6094, 8869,  580,
               3433, 5851, 9434, 5362, 7505, 7386, 8744, 9024, 2672, 2945, 8868,
               3444, 7390, 6909, 8991, 3462, 2310, 2033, 3406, 2279, 7146, 3956,
               5618,  238, 3757, 6028, 3289, 8843, 6362, 6319, 8565, 6490, 5497,
               7917, 4660, 8038, 7579, 9185, 8060, 7455, 5871, 7822,  392, 9057,
               2880, 2633, 9331, 9339, 4985, 1108, 5250, 7518, 3729, 5397, 3595,
               6931, 2590, 5466,  715, 5883, 1993, 3793, 7030, 2178, 4405, 6601,
               4256, 3177, 8057, 4064, 6404, 5483,  126, 8007, 8496,  381, 6044,
                 58, 9270, 1978, 4311, 1419,  267, 3368, 9291, 8931, 3992, 1204,
               8482, 7723, 5796, 8126, 2295, 1785, 9377, 7485, 2892, 3983, 2521,
               5233, 4834, 5509, 7301, 1178, 8461, 1608, 9575, 1549,  728,  243,
               6545, 2708, 3867, 1870,   77, 6559, 2228, 5321, 7051, 4374, 2933,
               6536, 2174, 4605, 7212, 8384, 9244, 3900, 1489, 5673, 7831, 2103,
               4526, 1904, 3444, 8088, 6887, 3093, 2941, 6450, 6222, 6366, 8768,
               7022,  549, 5682, 2372, 3785, 6496, 5489, 5475, 4519, 5376, 8735,
                330, 5123,  700, 7960, 6667,  648, 4949, 4048,  746, 3473,  610,
               4360,  575, 3696, 2807,  881, 4276, 7549, 1705, 3932, 3021, 9185,
               2809, 8297, 5636, 4468, 5981, 7667,   77, 5849, 4600, 1021,  196,
               8953, 2404, 2269, 3821, 5803]

In [7]:
data.shape

(9647, 29)

In [8]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-19,01:00:00,1,231,2,Wednesday,2015-08-19T01:00:00.000000000Z,1665,1.10380,1.10526,...,1,-0.00109,0.00037,0.00019,-0.00138,0.00052,1.103502,1.105358,0.001388,-0.000468
1,2015-08-19,05:00:00,5,231,2,Wednesday,2015-08-19T05:00:00.000000000Z,4532,1.10492,1.10734,...,1,-0.00046,0.00196,0.00009,-0.00109,-0.00138,1.103942,1.105107,0.001438,0.000273


# <font color='red'>CANDLE LOOP</font>

In [9]:
result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:

    #print('candle number is = ',candle_no)
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+12]
    #print('data is = ', data)
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'
        Current_Market_Stoploss= 0
    else:
        Current_Market = 'Bearish'
        Current_Market_Stoploss= 0 
    
    data = pd.read_csv(filename)
    data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C',
                          'SMA_10','SMA_20',
                          'f_time','julian_date','ATR_14',
#                          'col_1','col_2','col_3',
                          'col_4','col_5',
#                          'F_SMA_10','F_SMA_20'
                         ])

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+30]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY'
        else:
            Predicted_Trade = 'SELL'
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])

        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

In [10]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [11]:
result_output.head(30)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,5397.0,79.0,Bearish,0.0,4.0,BUY,3.0,SELL,53.0,SELL
1,5994.0,41.0,Bearish,0.0,31.0,BUY,17.0,BUY,25.0,SELL
2,2662.0,93.0,Bearish,0.0,76.0,BUY,82.0,BUY,8.0,SELL
3,5437.0,66.0,Bullish,0.0,61.0,BUY,19.0,SELL,55.0,BUY
4,6228.0,71.0,Bearish,0.0,65.0,BUY,11.0,SELL,20.0,BUY
5,9242.0,19.0,Bearish,0.0,15.0,SELL,52.0,SELL,18.0,SELL
6,5740.0,1.0,Bearish,0.0,54.0,BUY,42.0,BUY,43.0,BUY
7,706.0,89.0,Bullish,0.0,62.0,BUY,34.0,SELL,65.0,BUY
8,3041.0,6.0,Bearish,0.0,93.0,BUY,3.0,BUY,67.0,BUY
9,2241.0,0.0,Bearish,0.0,87.0,BUY,70.0,SELL,50.0,BUY
